<a href="https://colab.research.google.com/github/dhanushpachabhatla/My_LangChain_Playground/blob/main/langchain_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Agents Intro

In [1]:
!pip install langchain langsmith langchain_google_genai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 19.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
import os
from getpass import getpass

os.environ["LANGCHAIN_API_KEY"] = getpass("Enter LangSmith API Key: ")

# below should not be changed
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# you can change this as preferred
os.environ["LANGCHAIN_PROJECT"] = "pr-husky-bran-20"

Enter LangSmith API Key: ··········


In [66]:
import os
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ['GOOGLE_API_KEY'] = getpass("Enter your GEMINI KEY")

llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash',temperature= 0.2)

Enter your GEMINI KEY··········


# Tool Basics

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

In [4]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

In [5]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7edf7d54f100>)

In [7]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [8]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [9]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

In [10]:
import json

llm_output_string = "{\"x\": 6, \"y\": 3}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 6, 'y': 3}

This is then passed into the tool function as kwargs (keyword arguments) as indicated by the ** operator - the ** operator is used to unpack the dictionary into keyword arguments.

In [11]:
exponentiate.func(**llm_output_dict)

216

# Agent Creation

How It Works :

1. The agent reads your query.

2. It decides whether to use the Calculator tool or respond directly.

3. It uses `ReAct` (Reasoning + Acting) style prompting to determine steps.

- for now create Tool calling agent

- here we will have a scracthpad for llm to write down its thoughts while giving answer

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.agents import create_tool_calling_agent

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])


memory = ConversationBufferMemory(
    memory_key="history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)


tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

In [17]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'y': 7.68, 'x': 10.7}, log="\nInvoking: `multiply` with `{'y': 7.68, 'x': 10.7}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 7.68, "x": 10.7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--828a47bb-3eb5-4fae-8623-e5a7d390ae8e-0', tool_calls=[{'name': 'multiply', 'args': {'y': 7.68, 'x': 10.7}, 'id': '8141512d-ab05-4af2-8158-d27d8cc3da59', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 5, 'total_tokens': 96, 'input_token_details': {'cache_read': 0}})], tool_call_id='8141512d-ab05-4af2-8158-d27d8cc3da59')]

The LLM told us to use the multiply tool with inputs `{"x": 10.7, "y": 7.68}`. However, it didn't actually run the tool. For that, we need an agent execution loop, which manages the whole process of thinking, calling tools, and then thinking again. We'll use the `AgentExecutor` class to handle this for us.

In [18]:
from langchain.agents import AgentExecutor

agent_executer = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

Now let's try the same query with the executor, note that the intermediate_steps parameter that we added before is no longer needed as the executor handles it internally.

In [20]:
agent_executer.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 7.68, 'x': 10.7}`


82.17599999999999The answer is 82.176.


> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 82.176.\n', additional_kwargs={}, response_metadata={})],
 'output': 'The answer is 82.176.\n'}

We can see that the `multiply` tool was invoked, producing the observation of `82.175999...`. After the observation was provided, we can see that the LLM then generated a final response of:

```
10.7 multiplied by 7.68 is approximately 82.18.
```

This final response was generated based on the original query and the tool output (ie the _observation_). We can also confirm that this answer is accurate:

In [22]:
agent_executer.invoke({
    "input": "My name is James",
    "history": memory
})



> Entering new AgentExecutor chain...
Nice to meet you, James!  Is there anything I can help you with today?


> Finished chain.


{'input': 'My name is James',
 'history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 82.176.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James!  Is there anything I can help you with today?\n', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, James!  Is there anything I can help you with today?\n'}

In [23]:
agent_executer.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "history": memory
})



> Entering new AgentExecutor chain...
Here's how to solve that step-by-step:

1. **Multiplication:** 4 * 2 = 8
2. **Addition:** 9 + 10 = 19
3. **Subtraction:** 19 - 8 = 11
4. **Exponentiation:** 11³ = 1331

Therefore, the answer is $\boxed{1331}$


> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 82.176.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James!  Is there anything I can help you with today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Here's how to solve that step-by-step:\n\n1. **Multiplication:** 4 * 2 = 8\n2. **Addition:** 9 + 10 = 19\n3. **Subtraction:** 19 - 8 = 11\n4. **Exponentiation:** 11³ = 1331\n\nTherefore, the answer is $\\boxed{1331}$\n", additional_kwargs={}, response_metadata={})],
 'output': "Here's how to solve that step-by-step:\n\n1. **Multiplication:** 4 * 2 = 8

In [25]:
agent_executer.invoke({
    "input": "What is my name",
    "history": memory
})



> Entering new AgentExecutor chain...
Your name is James.


> Finished chain.


{'input': 'What is my name',
 'history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer is 82.176.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, James!  Is there anything I can help you with today?\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Here's how to solve that step-by-step:\n\n1. **Multiplication:** 4 * 2 = 8\n2. **Addition:** 9 + 10 = 19\n3. **Subtraction:** 19 - 8 = 11\n4. **Exponentiation:** 11³ = 1331\n\nTherefore, the answer is $\\boxed{1331}$\n", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Jam

# SerpAPI Weather Agent

In [28]:
!pip install -U langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [36]:
!pip install google-search-results --quiet

  Preparing metadata (setup.py) ... done


In [30]:
os.environ["SERPAPI_API_KEY"] = getpass("Enter your SerpAPI API key: ")

Enter your SerpAPI API key: ··········


In [43]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

method-1 : creating tool using `@tool`

In [48]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        print("raw data from ip - ",data)
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

In [49]:
tools = toolbox + [get_current_datetime, get_location_from_ip]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

In [50]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2025-09-02 14:04:20I am sorry, I cannot fulfill this request. I do not have the functionality to get your location and current weather.

> Finished chain.


#Basic calculator Agent

In [51]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.utilities import SerpAPIWrapper
import math

# Simple calculator tool
def calculator_tool(query: str) -> str:
    try:
        return str(eval(query))
    except:
        return "Error evaluating expression."

tools = [
    Tool(
        name="Calculator",
        func=calculator_tool,
        description="Evaluates mathematical expressions. Example: '2+2' or 'sqrt(16)'."
    ),
]

* initialize the agent

In [52]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

 Key Agent Types :

- ZERO_SHOT_REACT_DESCRIPTION – LLM decides from descriptions.

- CONVERSATIONAL_REACT_DESCRIPTION – Same but keeps chat history.

- STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION – For structured outputs.

In [34]:
response = agent.run("What is 45 * 12, and then add 100?")
print("Agent Response:", response)

/tmp/ipython-input-2425042886.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What is 45 * 12, and then add 100?")




> Entering new AgentExecutor chain...
Thought: I need to multiply 45 by 12 and then add 100 to the result.  I can use the calculator for this.
Action: Calculator
Action Input: 45*12+100
Observation: 640
Thought:Thought: I now know the final answer
Final Answer: 640

> Finished chain.
Agent Response: 640


# Realistic Example: Web Search + Calculator

In [55]:
search = SerpAPIWrapper()
tools.append(
    Tool(
        name="Web Search",
        func=search.run,
        description="Search the web for real-time information."
    )
)

In [67]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [68]:
response = agent.run("Find the population of France and multiply it by 2.")
print(response)



> Entering new AgentExecutor chain...
Action: Web Search
Action Input: population of France
Observation: {'type': 'population_result', 'population': '68.52 million', 'year': '2024'}
Thought:Action: Calculator
Action Input: 68.52 * 2
Observation: 137.04
Thought:Final Answer: The population of France multiplied by 2 is 137.04 million.

> Finished chain.
The population of France multiplied by 2 is 137.04 million.


#Conversation Multi Tool Agent

In [70]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.memory import ConversationBufferMemory

#use ConversationBufferMemory to store memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory, #pass it here
    verbose=True,
    handle_parsing_errors=True
)

response1 = agent.run("What is the population of India?  Give very recent & accurate data")
print("Response 1:", response1)

response2 = agent.run("Multiply that by 2 using Calculator.")
print("Response 2:", response2)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Web Search
Action Input: current population of India
Observation: {'type': 'population_result', 'population': '1.451 billion', 'year': '2024'}
Thought:Do I need to use a tool? No
AI: The current population of India is estimated to be 1.451 billion in 2024.

> Finished chain.
Response 1: The current population of India is estimated to be 1.451 billion in 2024.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 1.451 * 2
Observation: 2.902
Thought:Do I need to use a tool? No
AI: The population of India, multiplied by 2, is 2.902 billion.

> Finished chain.
Response 2: The population of India, multiplied by 2, is 2.902 billion.


How It Works :

1. First query fetches real-time data using `Web Search`.

2. Conversation `memory` stores that value.

3. Second query recalls previous answer and uses `Calculator`.

4. `verbose=True` shows agent reasoning.